In [ ]:
pip install dlib

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_dir = '/content/drive/My Drive/Drowsy/5.mov/'

# FINAL CODE

In [ ]:
import cv2
import dlib
import numpy as np
import zipfile
import os
import pickle


In [ ]:
# EAR Calculation
def eye_aspect_ratio(eye):
    x = [point.x for point in eye]
    y = [point.y for point in eye]
    A = np.linalg.norm(np.array([x[1] - x[5], y[1] - y[5]]))
    B = np.linalg.norm(np.array([x[2] - x[4], y[2] - y[4]]))
    C = np.linalg.norm(np.array([x[0] - x[3], y[0] - y[3]]))
    ear = (A + B) / (2.0 * C)
    return ear

In [ ]:
def pupil_to_eye_center_distance(eye):
    x = [point.x for point in eye]
    y = [point.y for point in eye]
    d = np.linalg.norm(np.array([x[0] - x[3], y[0] - y[3]]))
    return d

In [ ]:
def mouth_to_eye_ratio(eye, mouth):
    ear = eye_aspect_ratio(eye)
    mar = mouth_aspect_ratio(mouth)
    if ear == 0:
        return 0
    moe = mar / ear
    return moe

In [ ]:
# MAR Calculation
def mouth_aspect_ratio(mouth):
    x = [point.x for point in mouth]
    y = [point.y for point in mouth]
    A = np.linalg.norm(np.array([x[13] - x[19], y[13] - y[19]]))
    B = np.linalg.norm(np.array([x[14] - x[18], y[14] - y[18]]))
    C = np.linalg.norm(np.array([x[15] - x[17], y[15] - y[17]]))
    mar = (A + B + C) / (3.0 * np.linalg.norm(np.array([x[12] - x[16], y[12] - y[16]])))
    return mar

In [ ]:
# Extract EAR and MAR features
def extract_features(frame, detector, predictor):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    if len(faces) == 0:
        return [0, 0]  # Return zeros if no face is detected

    face = faces[0]
    landmarks = predictor(gray, face)

    # Indices based on the 68-point facial landmark model
    left_eye = [landmarks.part(i) for i in range(36, 42)]
    right_eye = [landmarks.part(i) for i in range(42, 48)]
    mouth = [landmarks.part(i) for i in range(48, 68)]

    # Calculate EAR and MAR
    ear_left = eye_aspect_ratio(left_eye)
    ear_right = eye_aspect_ratio(right_eye)
    ear = (ear_left + ear_right) / 2.0
    mar = mouth_aspect_ratio(mouth)

    return [ear, mar]


In [ ]:
# Load Dlib face detector and landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")


In [ ]:
# Function to determine drowsiness based on EAR and MAR
def determine_drowsiness(segment_features):
    threshold_ear = 0.28
    threshold_mar = 0.5
    result_str = ""

    for features in segment_features:
        ear, mar = features[:2]  # Using only EAR and MAR
        ear_status = "Below threshold" if ear < threshold_ear else "Above threshold"
        mar_status = "Above threshold" if mar > threshold_mar else "Below threshold"
        result_str += f"EAR: {ear:.2f} ({ear_status}), MAR: {mar:.2f} ({mar_status})\n"

    return result_str, 1 if ear_status == "Below threshold" and mar_status == "Above threshold" else 0

In [ ]:
# Access video from Google Drive
video_file_path = '/content/drive/My Drive/Drowsy/10.mov'
output_file = "drowsiness_features.pkl"

# Main Code Execution
all_features = []
all_labels = []


In [ ]:
# Function to process video and segment frames
def process_video_segments(video_path, detector, predictor, frames_per_segment=100, num_segments=30):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    if total_frames < frames_per_segment:
        cap.release()
        return [], []

    segment_positions = np.linspace(0, total_frames - frames_per_segment, num_segments, dtype=int)
    features_segments = []
    labels_segments = []

    for i, start_pos in enumerate(segment_positions):
        cap.set(cv2.CAP_PROP_POS_FRAMES, start_pos)
        current_segment = []

        for _ in range(frames_per_segment):
            ret, frame = cap.read()
            if not ret:
                break
            features = extract_features(frame, detector, predictor)
            current_segment.append(features)

        if len(current_segment) == frames_per_segment:
            print(f"\nSegment {i+1} Analysis:")
            segment_result, segment_label = determine_drowsiness(current_segment)
            print(segment_result)
            features_segments.append(current_segment)
            labels_segments.append(segment_label)

    cap.release()
    return features_segments, labels_segments

In [ ]:
if os.path.exists(video_file_path):
    print(f"Processing video: {video_file_path}")
    segments, labels = process_video_segments(video_file_path, detector, predictor)

    all_features.extend(segments)
    all_labels.extend(labels)

    print(f"Total segments collected: {len(all_features)}")
    for i, label in enumerate(all_labels):
        print(f"Segment {i+1}: {'Drowsy' if label == 1 else 'Not Drowsy'}")

    # Save features and labels
    data = {'features': np.array(all_features), 'labels': np.array(all_labels)}
    with open(output_file, 'wb') as f:
        pickle.dump(data, f)

    print(f"Features and labels saved to {output_file}")
else:
    print(f"Video file not found at {video_file_path}")

Processing video: /content/drive/My Drive/Drowsy/10.mov

Segment 1 Analysis:
EAR: 0.27 (Below threshold), MAR: 0.04 (Below threshold)
EAR: 0.28 (Above threshold), MAR: 0.03 (Below threshold)
EAR: 0.26 (Below threshold), MAR: 0.03 (Below threshold)
EAR: 0.26 (Below threshold), MAR: 0.04 (Below threshold)
EAR: 0.26 (Below threshold), MAR: 0.03 (Below threshold)
EAR: 0.28 (Above threshold), MAR: 0.03 (Below threshold)
EAR: 0.27 (Below threshold), MAR: 0.03 (Below threshold)
EAR: 0.29 (Above threshold), MAR: 0.02 (Below threshold)
EAR: 0.28 (Below threshold), MAR: 0.04 (Below threshold)
EAR: 0.28 (Above threshold), MAR: 0.03 (Below threshold)
EAR: 0.29 (Above threshold), MAR: 0.03 (Below threshold)
EAR: 0.31 (Above threshold), MAR: 0.03 (Below threshold)
EAR: 0.32 (Above threshold), MAR: 0.04 (Below threshold)
EAR: 0.29 (Above threshold), MAR: 0.04 (Below threshold)
EAR: 0.30 (Above threshold), MAR: 0.03 (Below threshold)
EAR: 0.31 (Above threshold), MAR: 0.04 (Below threshold)
EAR: 0.29 (

ANOTHER TEST